<a href="https://colab.research.google.com/github/pascuapablo/CEIA/blob/master/scripts/computer_vision/tp/TP_CVII_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras as k
import tensorflow.keras.models as models
import tensorflow.keras.layers as layers
import tensorflow.keras.optimizers as optimizers
from skimage.io import imread
import numpy as np
from random import sample
from keras.utils import to_categorical
import os
from glob import glob
from google.colab import drive


In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


## Funciones Helpers

In [ ]:
def preprocessing_image_ms(x, mean, std):
    # loop over image channels
    for idx, mean_value in enumerate(mean):
        x[..., idx] -= mean_value
        x[..., idx] /= std[idx]
    return x


def categorical_label_from_full_file_name(files, class_indices):
  
    # file basename without extension
    base_name = [os.path.splitext(os.path.basename(i))[0] for i in files]
    # class label from filename
    base_name = [i.split("_")[0] for i in base_name]
    # label to indices
    image_class = [class_indices[i] for i in base_name]
    # class indices to one-hot-label
    return to_categorical(image_class, num_classes=len(class_indices))

def hyperspectral_image_generator(files, class_indices, batch_size=32, image_mean=None,
                           rotation_range=0, shear_range=0, scale_range=1,
                           transform_range=0, horizontal_flip=False,
                           vertical_flip=False, crop=False, crop_size=None, filling_mode='edge',
                           speckle_noise=None):
    

    while True:
        # select batch_size number of samples without replacement
        batch_files = sample(files, batch_size)
        # get one_hot_label
        batch_Y = categorical_label_from_full_file_name(batch_files,
                                                        class_indices)
        # array for images
        batch_X = []
        # loop over images of the current batch
        for idx, input_path in enumerate(batch_files):
            normalization_factor =  0.0001
            image = np.array(imread(input_path), dtype=np.int16)
            image = image * normalization_factor
            
            # Imprime las imagenes que esta analizando
            # print(input_path, image.shape, image.max())
            
            if image_mean is not None:
                mean_std_data = np.loadtxt(image_mean, delimiter=',')
                image = preprocessing_image_ms(image, mean_std_data[0], mean_std_data[1])
            # process image
           
            # put all together
            batch_X += [image]
        # convert lists to np.array
        X = np.array(batch_X)
        Y = np.array(batch_Y)
       
        yield(X, Y)



## CNN

In [ ]:
# Carpeta del drive donde estan las imagenes
folderPath = '/content/drive/MyDrive/Cursos/CEIA/Computer Vision II/imagenes_tp'



files = np.random.permutation(glob(folderPath + "/**/*.tif"))

train_validation_split_index = np.ceil((len(files) * 0.7)).astype(int)

files_train = files[0: train_validation_split_index]
files_val = files[train_validation_split_index:]


BATCH_SIZE = 16
FILES_AMOUNT = len(files_train)
EPOCHS = 15



image_class = [f.split('/')[-1].split("_")[0] for f in files]
classes = {}
weight_clases = {}
for i,c in enumerate(np.unique(image_class)):
  classes[c]=i

  weight_clases[c]= np.sum(1 if ("/" + c + '/') in f else 0 for f in files)



train_generator = hyperspectral_image_generator(
    files_train.tolist(),
    class_indices = classes,
    batch_size=BATCH_SIZE
)


val_generator = hyperspectral_image_generator(
    files_val.tolist(),
    class_indices = classes,
    batch_size=1
)


CLASSES_COUNT = len(classes)

weight_clases


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


{'A': 5,
 'B': 6,
 'G': 1,
 'M': 210,
 'N': 78,
 'P': 54,
 'R': 6,
 'S': 342,
 'T': 1,
 'U': 12,
 'X': 18,
 'aa': 2,
 'm': 4,
 's': 87}

In [ ]:

from tensorflow.keras.metrics import CategoricalAccuracy
model = models.Sequential()

model.add(layers.Conv2D(64, (3,3), activation='relu', padding ='same',input_shape=(16,16,13)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(128, (3,3),  padding ='same', activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(256, (3,3),  padding ='same', activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(CLASSES_COUNT, activation='softmax'))

# compilar el modelo con binary_crossentropy y optimizador RMSprop con
# learning rate 1e-4, la métrica a usar es la accuracy (acc)

model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.Adam(lr=1e-4),
              metrics=[ CategoricalAccuracy(name="accuracy")] )

model.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_18 (Conv2D)           (None, 16, 16, 64)        7552      
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 8, 8, 64)          0         
_________________________________________________________________
conv2d_19 (Conv2D)           (None, 8, 8, 128)         73856     
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 4, 4, 128)         0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 4, 4, 256)         295168    
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 2, 2, 256)         0         
_________________________________________________________________
flatten_6 (Flatten)          (None, 1024)             

In [ ]:
model.fit_generator(
            train_generator,
            epochs=EPOCHS,
            steps_per_epoch=FILES_AMOUNT/BATCH_SIZE,
            validation_data=val_generator,
            validation_steps=5           
          )

Epoch 1/15
37/36 [==============================] - 2s 42ms/step - loss: 1.2591 - accuracy: 0.6064 - val_loss: 1.0287 - val_accuracy: 0.6000
Epoch 2/15
37/36 [==============================] - 2s 42ms/step - loss: 1.1977 - accuracy: 0.6182 - val_loss: 0.6020 - val_accuracy: 1.0000
Epoch 3/15
37/36 [==============================] - 2s 41ms/step - loss: 1.0946 - accuracy: 0.6402 - val_loss: 0.5736 - val_accuracy: 0.8000
Epoch 4/15
37/36 [==============================] - 2s 41ms/step - loss: 1.2741 - accuracy: 0.6064 - val_loss: 1.0311 - val_accuracy: 0.8000
Epoch 5/15
37/36 [==============================] - 2s 42ms/step - loss: 1.1577 - accuracy: 0.6605 - val_loss: 0.8503 - val_accuracy: 0.8000
Epoch 6/15
37/36 [==============================] - 2s 42ms/step - loss: 1.1400 - accuracy: 0.6470 - val_loss: 1.0792 - val_accuracy: 0.6000
Epoch 7/15
37/36 [==============================] - 2s 42ms/step - loss: 1.1515 - accuracy: 0.6436 - val_loss: 1.4260 - val_accuracy: 0.4000
Epoch 8/15
37

In [ ]:
y_predict = model.predict_generator(val_generator,steps=len(files_val))

y_true = categorical_label_from_full_file_name(files_val,classes)



In [ ]:
y_true_index = [ i.argmax() for i in y_true]
y_predict_index = [ i.argmax() for i in y_predict]

from sklearn.metrics import accuracy_score, balanced_accuracy_score


score = balanced_accuracy_score(y_true_index, y_predict_index)

b = (( np.array(y_true_index) - np.array(y_predict_index)))
c =(b == 0).sum() /len(y_true_index)
print(c,score)

0.3076923076923077 0.082109447444499
